In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import requests
import time
import io
import json
import re

In [ ]:
def auth_headers(token):
    return {'Authorization': 'OAuth {token}'.format(token=token)}

### Выгрузка из Аппметрики

In [ ]:
responses = []
token = "YOUR_TOKEN"
apikey = "YOUR_APIKEY"

In [ ]:
date_since = '2019-05-23'
date_until = '2019-05-23'
columns_to_load = ['event_name', 'event_timestamp', 'appmetrica_device_id', 'os_name']
columns_to_extract = ['event_name', 'event_timestamp', 'appmetrica_device_id', 'os_name']
columns_to_load = str(columns_to_load).replace("', '", "%2C").replace("['", '').replace("']", '')
operating_systems = ['ios']

в app_details_url лежит сам запрос, поля указываются в конце запроса через запятую (%2C)

In [ ]:
def appmetrica_loader(apikey, date_since, date_until, token):
    app_details_url = """https://api.appmetrica.yandex.ru/logs/v1/export/events.csv?application_id={apikey}&date_since={date_since}%2000%3A00%3A00&date_until={date_until}%2023%3A59%3A59&date_dimension=default&use_utf8_bom=true&fields={columns_to_load}""".format(apikey=apikey
                                                                                 , date_until=date_until
                                                                                 , date_since=date_since
                                                                                 , columns_to_load=columns_to_load)
    
    while True:
        r = requests.get(app_details_url, headers=auth_headers(token))
        if r.status_code == 200:
            s=requests.get(app_details_url, headers=auth_headers(token)).content
            df=pd.read_csv(io.StringIO(s.decode('utf-8')))
            return(df)
            break;
        if r.status_code != 202:
            print('Bad response code')
            exit()
        print('Result is not ready yet. Waiting')
       

In [ ]:
def filter_and_convert(df, my_id):
    mydf = df[df['appmetrica_device_id'] == my_id].sort_values(by='event_timestamp')  
    mydf['event_timestamp'] = pd.to_datetime(mydf['event_timestamp'] * (10 ** 9))  # convert timestamps, would be better to use _convert_timestamps from utils
    return mydf

def to_seconds(dt):
    return int((dt - pd.datetime(1970,1,1)).total_seconds())


каждый раз надо указывать новый временной промежуток, потому что на повторный запрос аппметрика будет отправлять предыдущий ответ

In [ ]:
df = appmetrica_loader(apikey, date_since, date_until, token)

In [ ]:
df

### Выгрузка из Метрики

In [ ]:
def auth_headers(token):
    return {'Authorization': 'OAuth {token}'.format(token=token)}

In [ ]:
id_app = "YOUR_COUNTER" #номер счетчика
token_m = "YOUR_TOKEN" #отладочный токен

#### Для аггрегированных отчетов

In [ ]:
import requests
import sys

header = {'Authorization': 'OAuth {token}'.format(token=token_m)}
ids = {
    'Counter': id_app,
}

payload = {
    'metrics': 'ym:s:visits,ym:s:users', #важно сохранить такой формат записи
    'date1': '2019-07-16',
    'date2': '2019-07-18',
    #'filters': "ym:s:deviceCategory!='mobile'",
    'ids': id_app,
    'accuracy': 'full',
    'group': 'day',
    'expand': True,
    'pretty': True,
}

i = 1
for key, value in ids.items():
    payload['ids'] = value
    r = requests.get('https://api-metrika.yandex.ru/stat/v1/data/drilldown', params=payload, headers=header)
    data = str(r.json()).split(",")
    i += 1
    payload['ids'] = value
    print('total', key, data)

#### Для сырых логов

###### Названия нужных полей можно взять отсюда:

https://yandex.ru/dev/metrika/doc/api2/logs/fields/hits-docpage/


https://yandex.ru/dev/metrika/doc/api2/logs/fields/visits-docpage/

In [ ]:
import requests
import sys

header = {'Authorization': 'OAuth {token}'.format(token=token_m)}
ids = {
    'Counter': id_app,
}
#Параметры
payload = {
    'fields': 'ym:pv:dateTime,ym:pv:watchID',
    'date1': '2019-07-16',
    'date2': '2019-07-17',
    'source': 'hits' # another option is 'visits'
}


In [ ]:
from time import sleep

#Функция для отправки запроса и выгрузки
def get_results(payload, id_app):
    print('It may take a while')
    p = requests.post('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequests'.format(id_app=id_app), params=payload, headers=header).content
    request_id = json.load(io.StringIO(p.decode('utf-8')))['log_request']['request_id']
    idx = 1
    while True:
        print(f"Query: {idx}")
        r = requests.get('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequest/{request_id}/part/0/download'.format(id_app=id_app, request_id=request_id), headers=header).content
        if pd.read_csv(io.StringIO(r.decode('utf-8')), sep='\t').shape[0] == 0:
            idx += 1
            sleep(60)
        else:
            break
    data = pd.read_csv(io.StringIO(r.decode('utf-8')), sep='\t')
    return data


In [ ]:
#Выгрузка
get_results(payload, id_app)

Запасной вариант, вручную то же самое

In [ ]:
#Отправляем запрос на создание выгрузки, нам нужно забрать request_id, его подставляем в следующий запрос
import json
p = requests.post('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequests'.format(id_app=id_app), params=payload, headers=header).content
request_id = json.load(io.StringIO(p.decode('utf-8')))['log_request']['request_id']

In [ ]:
r = requests.get('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequest/{request_id}/part/0/download'.format(id_app=id_app, request_id=request_id), headers=header).content

In [ ]:
data = pd.read_csv(io.StringIO(r.decode('utf-8')), sep='\t')